<a href="https://colab.research.google.com/github/Blessedtomato/Megastore-Sales/blob/main/Toto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Xisiqomelir/xisiqo_datadump/main/ToTo.csv")

In [3]:
df.head()

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number,From Last,...,Division 3 Winners,Division 3 Prize,Division 4 Winners,Division 4 Prize,Division 5 Winners,Division 5 Prize,Division 6 Winners,Division 6 Prize,Division 7 Winners,Division 7 Prize
0,3884,2023-07-06,13,16,28,34,38,40,47,"34,40",...,147.0,1666.0,263.0,508.0,6591.0,50.0,7890.0,25.0,121282.0,10.0
1,3883,2023-07-03,14,30,34,40,45,49,12,30,...,85.0,2223.0,235.0,439.0,4664.0,50.0,6423.0,25.0,87982.0,10.0
2,3882,2023-06-29,5,15,16,30,37,46,49,5,...,515.0,1661.0,1208.0,387.0,25404.0,50.0,33315.0,25.0,475093.0,10.0
3,3881,2023-06-26,4,5,17,28,39,44,31,28,...,237.0,1741.0,531.0,424.0,12396.0,50.0,16290.0,25.0,232715.0,10.0
4,3880,2023-06-22,3,9,28,32,37,46,7,"28,32,46",...,112.0,2163.0,358.0,369.0,6596.0,50.0,11867.0,25.0,125106.0,10.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Draw                1544 non-null   int64  
 1   Date                1544 non-null   object 
 2   Winning Number 1    1544 non-null   int64  
 3   2                   1544 non-null   int64  
 4   3                   1544 non-null   int64  
 5   4                   1544 non-null   int64  
 6   5                   1544 non-null   int64  
 7   6                   1544 non-null   int64  
 8   Additional Number   1544 non-null   int64  
 9   From Last           915 non-null    object 
 10  Same As Day         208 non-null    float64
 11  Odd                 1544 non-null   int64  
 12  Even                1544 non-null   int64  
 13   1-10               1544 non-null   int64  
 14   11-20              1544 non-null   int64  
 15   21-30              1544 non-null   int64  
 16   31-40

In [5]:
df.drop(['From Last', 'Same As Day', 'Odd', 'Even', ' 1-10', ' 11-20', ' 21-30', ' 31-40', ' 41-50', 'Division 1 Winners', 'Division 1 Prize', 'Division 2 Winners', 'Division 2 Prize', 'Division 3 Winners', 'Division 3 Prize', 'Division 4 Winners', 'Division 4 Prize', 'Division 5 Winners', 'Division 5 Prize', 'Division 6 Winners', 'Division 6 Prize', 'Division 7 Winners', 'Division 7 Prize'], axis=1, inplace=True)
df.head()

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number
0,3884,2023-07-06,13,16,28,34,38,40,47
1,3883,2023-07-03,14,30,34,40,45,49,12
2,3882,2023-06-29,5,15,16,30,37,46,49
3,3881,2023-06-26,4,5,17,28,39,44,31
4,3880,2023-06-22,3,9,28,32,37,46,7


In [6]:
df.drop(['Date', 'Draw'], axis=1, inplace=True)
df.head()

,Winning Number 1,2,3,4,5,6,Additional Number
0,13,16,28,34,38,40,47
1,14,30,34,40,45,49,12
2,5,15,16,30,37,46,49
3,4,5,17,28,39,44,31
4,3,9,28,32,37,46,7


In [7]:
df.drop(['Additional Number '], axis=1, inplace=True)
df.head()

,Winning Number 1,2,3,4,5,6
0,13,16,28,34,38,40
1,14,30,34,40,45,49
2,5,15,16,30,37,46
3,4,5,17,28,39,44
4,3,9,28,32,37,46


In [8]:
df = df[:-654]
#To remove draws before 9/10/2014 because they are 45 numbers before

In [9]:
print(df.shape)

(890, 6)


In [10]:
df.describe()

,Winning Number 1,2,3,4,5,6
count,890.000000,890.000000,890.000000,890.000000,890.000000,890.000000
mean,6.823596,14.055056,21.270787,28.601124,35.632584,43.100000
std,5.564764,7.350941,7.960090,8.061867,7.301236,5.420257
min,1.000000,2.000000,4.000000,7.000000,14.000000,19.000000
25%,3.000000,9.000000,15.000000,23.000000,31.000000,40.000000
50%,5.000000,13.000000,21.000000,29.000000,37.000000,45.000000
75%,9.000000,19.000000,27.000000,35.000000,41.000000,47.000000
max,33.000000,38.000000,43.000000,47.000000,48.000000,49.000000


In [11]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [12]:
transformed_df.head()

,0,1,2,3,4,5
0,1.110537,0.264733,0.845844,0.670057,0.324431,-0.572250
1,1.290340,2.170322,1.600028,1.414720,1.283712,1.089121
2,-0.327888,0.128620,-0.662524,0.173615,0.187391,0.535331
3,-0.507691,-1.232515,-0.536827,-0.074606,0.461471,0.166137
4,-0.687495,-0.688061,0.845844,0.421836,0.187391,0.535331


In [13]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

890

In [14]:
# Amount of games we need to take into consideration for prediction
window_length = 50
window_length

50

In [15]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

6

In [16]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [17]:
X.shape

(840, 50, 6)

In [18]:
y.shape

(840, 6)

In [19]:
X[0]

array([[ 1.11053721,  0.2647331 ,  0.84584431,  0.6700572 ,  0.32443094,
        -0.57225014],
       [ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123],
       [-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
        -1.31063742],
       [-0.1480851 ,  0.94530056,  1.22293637,  0.42183622, -0.08668942,
         0.90452441],
       [-0.32788828,  0.2647331 ,  0.59444961, -0.07460574, -0.49780979,
        -1.67983105],
       [ 0.03171809,  0.1286196 ,  1.47433107,  1.

In [20]:
y[0]

array([-0.86729784, -1.09640183, -1.66810272, -0.07460574, -0.63484991,
       -1.1260406 ])

In [21]:
X[1]

array([[ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123],
       [-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
        -1.31063742],
       [-0.1480851 ,  0.94530056,  1.22293637,  0.42183622, -0.08668942,
         0.90452441],
       [-0.32788828,  0.2647331 ,  0.59444961, -0.07460574, -0.49780979,
        -1.67983105],
       [ 0.03171809,  0.1286196 ,  1.47433107,  1.41472015,  1.28371178,
         1.08912123],
       [-0.32788828, -0.14360738,  1.09723902,  0.

In [22]:
y[1]

array([-1.04710103, -1.50474231, -1.03961596, -1.68804213, -1.86821099,
       -0.01845968])

In [23]:
model = Sequential()

In [24]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [25]:
model.add(Dropout(0.2))

In [26]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [27]:
model.add(Dropout(0.2))

In [28]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [29]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

In [30]:
model.add(Dense(59))

In [31]:
model.add(Dense(number_of_features))

In [32]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [33]:
model.fit(x=X, y=y, batch_size=600, epochs=600, verbose=2)

Epoch 1/600
2/2 - 57s - loss: 0.9964 - accuracy: 0.1833 - 57s/epoch - 29s/step
Epoch 2/600
2/2 - 27s - loss: 0.9926 - accuracy: 0.1917 - 27s/epoch - 14s/step
Epoch 3/600
2/2 - 30s - loss: 0.9909 - accuracy: 0.1988 - 30s/epoch - 15s/step
Epoch 4/600
2/2 - 27s - loss: 0.9888 - accuracy: 0.2060 - 27s/epoch - 14s/step
Epoch 5/600
2/2 - 27s - loss: 0.9873 - accuracy: 0.2000 - 27s/epoch - 13s/step
Epoch 6/600
2/2 - 27s - loss: 0.9863 - accuracy: 0.2000 - 27s/epoch - 13s/step
Epoch 7/600
2/2 - 27s - loss: 0.9851 - accuracy: 0.1929 - 27s/epoch - 14s/step
Epoch 8/600
2/2 - 28s - loss: 0.9836 - accuracy: 0.1917 - 28s/epoch - 14s/step
Epoch 9/600
2/2 - 27s - loss: 0.9822 - accuracy: 0.2036 - 27s/epoch - 13s/step
Epoch 10/600
2/2 - 27s - loss: 0.9818 - accuracy: 0.1952 - 27s/epoch - 13s/step
Epoch 11/600
2/2 - 27s - loss: 0.9803 - accuracy: 0.2060 - 27s/epoch - 14s/step
Epoch 12/600
2/2 - 27s - loss: 0.9797 - accuracy: 0.1988 - 27s/epoch - 14s/step
Epoch 13/600
2/2 - 27s - loss: 0.9781 - accuracy:

In [50]:
to_predict = df.head(15)
to_predict

,Winning Number 1,2,3,4,5,6
0,13,16,28,34,38,40
1,14,30,34,40,45,49
2,5,15,16,30,37,46
3,4,5,17,28,39,44
4,3,9,28,32,37,46
5,2,25,28,30,32,46
6,6,15,17,42,46,49
7,2,22,28,33,34,36
8,6,21,31,32,35,48
9,5,16,26,28,32,34


In [51]:
to_predict = np.array(to_predict)
to_predict

array([[13, 16, 28, 34, 38, 40],
       [14, 30, 34, 40, 45, 49],
       [ 5, 15, 16, 30, 37, 46],
       [ 4,  5, 17, 28, 39, 44],
       [ 3,  9, 28, 32, 37, 46],
       [ 2, 25, 28, 30, 32, 46],
       [ 6, 15, 17, 42, 46, 49],
       [ 2, 22, 28, 33, 34, 36],
       [ 6, 21, 31, 32, 35, 48],
       [ 5, 16, 26, 28, 32, 34],
       [ 7, 15, 33, 40, 45, 49],
       [ 5, 13, 30, 34, 36, 48],
       [ 4, 15, 17, 35, 42, 48],
       [ 1,  4,  7,  8, 34, 44],
       [ 7, 13, 18, 19, 25, 29]])

In [52]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

array([[ 1.11053721,  0.2647331 ,  0.84584431,  0.6700572 ,  0.32443094,
        -0.57225014],
       [ 1.29034039,  2.17032199,  1.60002843,  1.41472015,  1.28371178,
         1.08912123],
       [-0.32788828,  0.1286196 , -0.66252391,  0.17361524,  0.18739082,
         0.53533078],
       [-0.50769147, -1.23251532, -0.53682656, -0.07460574,  0.46147106,
         0.16613714],
       [-0.68749466, -0.68806135,  0.84584431,  0.42183622,  0.18739082,
         0.53533078],
       [-0.86729784,  1.48975453,  0.84584431,  0.17361524, -0.49780979,
         0.53533078],
       [-0.1480851 ,  0.1286196 , -0.53682656,  1.66294113,  1.4207519 ,
         1.08912123],
       [-0.86729784,  1.08141405,  0.84584431,  0.54594671, -0.22372954,
        -1.31063742],
       [-0.1480851 ,  0.94530056,  1.22293637,  0.42183622, -0.08668942,
         0.90452441],
       [-0.32788828,  0.2647331 ,  0.59444961, -0.07460574, -0.49780979,
        -1.67983105],
       [ 0.03171809,  0.1286196 ,  1.47433107,  1.

In [53]:
y_pred = model.predict(np.array([scaled_to_predict]))
print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 68ms/step
The predicted numbers in the last lottery game are: [23 25 25 29 25 39]
